In [1]:
import os
os.getcwd()

%pip install -e .. -qqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
from pathlib import Path
from utils import load_run
import plotly.express as px

%load_ext autoreload
%autoreload 2

In [3]:
ROOT = Path.cwd().parent

MODEL_PATH         = ROOT / "model" / "resnet_final.keras"
SNOMED_CLASSES_NPY = ROOT / "data" / "snomed_classes.npy"
SEL_DF_CSV         = ROOT / "data" / "ecg_model_pred_data.csv"

ECG_FILENAMES_PATH = ROOT / "data" / "ecg_filenames.npy"
PROBS_PATH         = ROOT / "data" / "ecg_model_probs.npy"
CLASS_NAMES_PATH   = ROOT / "data" / "snomed_classes.npy"
Y_TRUE_PATH        = ROOT / "data" / "ecg_y_true.npy"
ECG_DURATIONS_PATH = ROOT / "data" / "ecg_durations.npy"
ECG_PRED_PATH      = ROOT / "data" / "ecg_xai_sel_meta_p0.85_k5.csv"


In [4]:
# ---- choose mode ----
run_mode = "eval"   # "eval" or "demo"

MODE_CFG = {
    "eval": {"max_examples_per_class": 50, "plot": False},
    "demo": {"max_examples_per_class": 3,  "plot": False},
}

In [5]:
OUT_BASE = ROOT / "outputs"
RUN_DIR = OUT_BASE / run_mode

cfg = MODE_CFG[run_mode]

all_fused_payloads, df_lime_all, df_ts_all, sel_df = load_run(RUN_DIR)

json_str = json.dumps(all_fused_payloads)
with open(RUN_DIR/"df_fused_all.json", "w", encoding="utf-8") as f:
    obj = json.loads(json_str)
    json.dump(obj, f, indent=2, ensure_ascii=False)

df_lime_all.to_csv(RUN_DIR / "df_lime_all.csv", index=False)
df_ts_all.to_csv(RUN_DIR / "df_ts_all.csv", index=False)

In [6]:
import json
import numpy as np
import pandas as pd

def _safe_json_load(x):
    """Parse JSON stored as string; return None if empty."""
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    if isinstance(x, (list, dict)):
        return x
    s = str(x).strip()
    if not s or s.lower() in {"nan", "none"}:
        return None
    return json.loads(s)

def _segments_to_starts_ends(segments):
    """
    segments_json is expected to represent N segments.
    Handles common shapes:
      - [[s0,e0], [s1,e1], ...]
      - [{"start":s,"end":e}, ...]
      - [{"s":s,"e":e}, ...]
    Returns list of (start, end) as ints.
    """
    if segments is None:
        return None

    out = []
    for seg in segments:
        if isinstance(seg, (list, tuple)) and len(seg) >= 2:
            out.append((int(seg[0]), int(seg[1])))
        elif isinstance(seg, dict):
            if "start" in seg and "end" in seg:
                out.append((int(seg["start"]), int(seg["end"])))
            elif "s" in seg and "e" in seg:
                out.append((int(seg["s"]), int(seg["e"])))
            else:
                # last resort: try first two values
                vals = list(seg.values())
                if len(vals) >= 2:
                    out.append((int(vals[0]), int(vals[1])))
                else:
                    raise ValueError(f"Unrecognized segment dict: {seg}")
        else:
            raise ValueError(f"Unrecognized segment entry: {seg}")
    return out

def to_long_segments(
    df: pd.DataFrame,
    *,
    method: str,
    scores_json_col: str,
    segments_json_col: str = "segments_json",
    key_cols=("mat_path", "target_class_explained", "group_class", "val_idx", "fs", "window_sec")
) -> pd.DataFrame:
    """
    Convert a per-record CSV (LIME/TS) into long form per segment.
    """
    rows = []

    # ensure key cols exist if referenced
    for c in key_cols:
        if c not in df.columns:
            # don't fail hard; just skip missing keys
            pass

    for _, r in df.iterrows():
        scores = _safe_json_load(r.get(scores_json_col))
        segs = _segments_to_starts_ends(_safe_json_load(r.get(segments_json_col)))

        if scores is None or segs is None:
            continue

        # scores should be length N
        if not isinstance(scores, list):
            raise ValueError(f"{scores_json_col} should be a JSON list. Got: {type(scores)}")
        if len(scores) != len(segs):
            # if this happens, we drop it rather than forcing alignment
            continue

        base = {c: r.get(c) for c in key_cols if c in df.columns}
        base["method"] = method
        base["num_segments"] = int(r.get("num_segments", len(segs)))

        # build per-segment rows
        abs_scores = np.abs(np.asarray(scores, dtype=float))
        denom = float(abs_scores.sum()) if abs_scores.sum() > 0 else 1.0

        for i, ((s, e), sc) in enumerate(zip(segs, scores)):
            rows.append({
                **base,
                "segment_idx": int(i),
                "seg_start": int(s),
                "seg_end": int(e),
                "score": float(sc),
                "score_abs": float(abs(float(sc))),
                "score_abs_norm": float(abs(float(sc)) / denom),
            })

    return pd.DataFrame(rows)


In [7]:
df_ts = pd.read_csv(RUN_DIR / "df_ts_all.csv")
df_lime = pd.read_csv(RUN_DIR / "df_lime_all.csv")

# --- Build the common long DF ---
df_ts_long = to_long_segments(
    df_ts,
    method="TimeSHAP",
    scores_json_col="timeshap_event_values_json",
)

df_lime_long = to_long_segments(
    df_lime,
    method="LIME",
    scores_json_col="event_importances_json",
)

df_long = pd.concat([df_lime_long, df_ts_long], ignore_index=True)

# (optional) normalise types for joining/grouping
df_long["mat_path"] = df_long["mat_path"].astype(str)
df_long["target_class_explained"] = df_long["target_class_explained"].astype(str)

df_long.head()

,mat_path,target_class_explained,group_class,val_idx,fs,window_sec,method,num_segments,segment_idx,seg_start,seg_end,score,score_abs,score_abs_norm
0,C:\UHull\Data\Training_2\Q0946.mat,1,164889003,1992,500.0,0.5,LIME,20,0,0,250,-0.018132,0.018132,0.019340
1,C:\UHull\Data\Training_2\Q0946.mat,1,164889003,1992,500.0,0.5,LIME,20,1,250,500,0.093533,0.093533,0.099766
2,C:\UHull\Data\Training_2\Q0946.mat,1,164889003,1992,500.0,0.5,LIME,20,2,500,750,0.020973,0.020973,0.022370
3,C:\UHull\Data\Training_2\Q0946.mat,1,164889003,1992,500.0,0.5,LIME,20,3,750,1000,0.057867,0.057867,0.061724
4,C:\UHull\Data\Training_2\Q0946.mat,1,164889003,1992,500.0,0.5,LIME,20,4,1000,1250,0.054820,0.054820,0.058473


In [8]:
coverage = (
    df_long.groupby(["mat_path", "target_class_explained", "segment_idx"])["method"]
    .nunique()
    .value_counts()
)

coverage


method
2    3000
Name: count, dtype: int64

In [9]:
import plotly.express as px

df_mean = (
    df_long.groupby(["target_class_explained", "method", "segment_idx"], as_index=False)
    .agg(mean_abs_norm=("score_abs_norm", "mean"))
)

fig = px.line(
    df_mean,
    x="segment_idx",
    y="mean_abs_norm",
    color="method",
    facet_col="target_class_explained",
    markers=True,
    title="Mean normalised segment importance by method (LIME vs TimeSHAP)",
)
fig.show()


In [10]:
# Pivot to compare methods side-by-side
df_piv = (
    df_long.pivot_table(
        index=["mat_path", "target_class_explained", "segment_idx"],
        columns="method",
        values="score_abs_norm",
        aggfunc="mean"
    )
    .reset_index()
)

# Keep only where both are present
df_piv = df_piv.dropna(subset=["LIME", "TimeSHAP"])

fig = px.scatter(
    df_piv,
    x="LIME",
    y="TimeSHAP",
    facet_col="target_class_explained",
    opacity=0.35,
    title="Segment-wise agreement (abs-norm): LIME vs TimeSHAP",
    trendline="ols",
)
fig.show()


In [11]:
import numpy as np
import plotly.express as px

# only keep rows where both exist
dfp = df_piv.dropna(subset=["LIME", "TimeSHAP"]).copy()

df_spear = (
    dfp.groupby(["mat_path", "target_class_explained"])[["LIME", "TimeSHAP"]]
       .apply(lambda g: g["LIME"].corr(g["TimeSHAP"], method="spearman") if len(g) >= 3 else np.nan)
       .reset_index(name="spearman")
)

fig = px.box(
    df_spear.dropna(subset=["spearman"]),
    x="target_class_explained",
    y="spearman",
    points="all",
    title="Per-record Spearman agreement between LIME and TimeSHAP (segment abs-norm)",
)
fig.show()



In [12]:
import json
import numpy as np
import pandas as pd

LEADS12 = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]

df_ts = pd.read_csv(RUN_DIR / "df_ts_all.csv")
df_lime = pd.read_csv(RUN_DIR / "df_lime_all.csv")

# Make sure join keys are consistent
for df in (df_lime, df_ts):
    df["val_idx"] = pd.to_numeric(df["val_idx"], errors="coerce").astype("Int64")
    df["target_class_explained"] = df["target_class_explained"].astype(str)
    df["mat_path"] = df["mat_path"].astype(str)

def _safe_json_load(s):
    if pd.isna(s) or s is None or str(s).strip() == "":
        return {}
    if isinstance(s, (dict, list)):
        return s
    return json.loads(s)

def lead_mass_from_top5(perlead_top5_json: str) -> dict:
    """
    perlead_top5_json format (as you pasted):
      {"6": [[start, end, w], ...], "7": [...], ...}
    Returns: {lead_idx(int): mass(float)} where mass = sum(|w| * duration)
    """
    d = _safe_json_load(perlead_top5_json)
    out = {i: 0.0 for i in range(12)}
    for k, spans in (d or {}).items():
        li = int(k)
        m = 0.0
        for s,e,w in spans:
            dur = float(e) - float(s)
            m += abs(float(w)) * max(dur, 0.0)
        if 0 <= li < 12:
            out[li] += m
    return out

def to_long_leads(df: pd.DataFrame, method_name: str, col_json: str) -> pd.DataFrame:
    rows = []
    for _, r in df.iterrows():
        masses = lead_mass_from_top5(r[col_json])
        total = sum(masses.values())
        for li in range(12):
            rows.append({
                "val_idx": r["val_idx"],
                "mat_path": r["mat_path"],
                "target_class_explained": r["target_class_explained"],
                "method": method_name,
                "lead_idx": li,
                "lead": LEADS12[li],
                "lead_mass": float(masses[li]),
                "lead_mass_norm": float(masses[li] / total) if total > 0 else 0.0,
            })
    return pd.DataFrame(rows)

df_long_lime = to_long_leads(df_lime, "LIME", "perlead_spans_top5_json")
df_long_ts   = to_long_leads(df_ts,   "TimeSHAP", "perlead_timeshap_top5_json")

df_lead = pd.concat([df_long_lime, df_long_ts], ignore_index=True)

# Optional: keep only rows where both methods exist for the record+class
pairs = (
    df_lead.groupby(["val_idx","mat_path","target_class_explained"])["method"]
    .nunique()
    .reset_index(name="n_methods")
)
df_lead = df_lead.merge(pairs, on=["val_idx","mat_path","target_class_explained"], how="left")
df_lead = df_lead[df_lead["n_methods"] == 2].drop(columns=["n_methods"])

df_lead.head()


,val_idx,mat_path,target_class_explained,method,lead_idx,lead,lead_mass,lead_mass_norm
0,1992,C:\UHull\Data\Training_2\Q0946.mat,1,LIME,0,I,0.000326,0.037587
1,1992,C:\UHull\Data\Training_2\Q0946.mat,1,LIME,1,II,0.000752,0.086668
2,1992,C:\UHull\Data\Training_2\Q0946.mat,1,LIME,2,III,0.000000,0.000000
3,1992,C:\UHull\Data\Training_2\Q0946.mat,1,LIME,3,aVR,0.000000,0.000000
4,1992,C:\UHull\Data\Training_2\Q0946.mat,1,LIME,4,aVL,0.000000,0.000000


In [13]:
import plotly.express as px

df_prof = (
    df_lead.groupby(["target_class_explained","method","lead"], as_index=False)
    .agg(mean_norm=("lead_mass_norm","mean"),
         med_norm=("lead_mass_norm","median"))
)

fig = px.line(
    df_prof,
    x="lead",
    y="mean_norm",
    color="method",
    facet_col="target_class_explained",
    markers=True,
    title="Lead importance profile (mean of abs-duration-normalised mass)"
)
fig.update_layout(legend_title_text="Method")
fig.show()


In [14]:
from scipy.stats import spearmanr

# pivot to 12-d lead vectors per record+class
df_vec = (
    df_lead.pivot_table(
        index=["val_idx","mat_path","target_class_explained","lead"],
        columns="method",
        values="lead_mass_norm",
        aggfunc="mean",
        fill_value=0.0
    )
    .reset_index()
)

def _spearman_group(g):
    if g["LIME"].nunique() < 2 and g["TimeSHAP"].nunique() < 2:
        return np.nan
    return spearmanr(g["LIME"], g["TimeSHAP"]).correlation

df_spear = (
    df_vec.groupby(["val_idx","mat_path","target_class_explained"], as_index=False)
    .apply(lambda g: pd.Series({"spearman": _spearman_group(g)}), include_groups=False)
)

fig = px.box(
    df_spear.dropna(),
    x="target_class_explained",
    y="spearman",
    points="all",
    title="Per-record lead agreement (Spearman): LIME vs TimeSHAP"
)
fig.show()


In [15]:
df_eval_stability = pd.read_csv(ROOT / "data" / "df_eval_stability.csv")

df_eval_stability[['class_name','extra_mid_k_eff']].describe()

,extra_mid_k_eff
count,100.000000
mean,13.980000
std,3.749694
min,10.000000
25%,10.000000
50%,13.000000
75%,17.000000
max,20.000000


In [16]:
from pathlib import Path
import pandas as pd
import plotly.express as px

# -----------------------------
# CONFIG (edit these paths)
# -----------------------------
EVAL_CSV = ROOT / "data" / "df_eval_attauc_deletion.csv"
STAB_CSV = ROOT / "data" / "df_eval_stability.csv"
OUTDIR = Path("outputs/plots_af_snr")
OUTDIR.mkdir(parents=True, exist_ok=True)

# Your mapping (if you prefer short labels)
CLASS_LABELS = {
    "atrial fibrillation": "AF",
    "sinus rhythm": "SNR",
    "164889003": "AF",
    "426783006": "SNR",
}

def _label_class(row):
    # Prefer class_name if present; fall back to meta_code
    cn = str(row.get("class_name", "")).strip().lower()
    mc = str(row.get("meta_code", "")).strip()
    if cn == "atrial fibrillation":
        return "AF"
    if cn == "sinus rhythm":
        return "SNR"
    if mc == "164889003":
        return "AF"
    if mc == "426783006":
        return "SNR"
    return row.get("class_name", row.get("meta_code", "Unknown"))

def _maybe_write_png(fig, path_png: Path):
    """
    Save PNG if kaleido is available. Otherwise skip silently.
    """
    try:
        fig.write_image(str(path_png), scale=2)
    except Exception as e:
        print(f"[INFO] PNG export skipped (install kaleido to enable): {e}")

# -----------------------------
# LOAD + FILTER
# -----------------------------
df_eval = pd.read_csv(EVAL_CSV)
df_stab = pd.read_csv(STAB_CSV)

# Normalise types
for df in (df_eval, df_stab):
    if "meta_code" in df.columns:
        df["meta_code"] = df["meta_code"].astype(str)
    if "class_name" in df.columns:
        df["class_name"] = df["class_name"].astype(str)

# Keep AF + SNR only
keep_names = {"atrial fibrillation", "sinus rhythm"}
df_eval = df_eval[df_eval["class_name"].str.lower().isin(keep_names)].copy()
df_stab = df_stab[df_stab["class_name"].str.lower().isin(keep_names)].copy()

df_eval["class_short"] = df_eval.apply(_label_class, axis=1)
df_stab["class_short"] = df_stab.apply(_label_class, axis=1)

# Optional: order
class_order = ["AF", "SNR"]

# -----------------------------
# PLOT 1 — AttAUC (lenient)
# -----------------------------
fig1 = px.box(
    df_eval.dropna(subset=["lenient_attauc"]),
    x="class_short",
    y="lenient_attauc",
    points="all",
    category_orders={"class_short": class_order},
    hover_data=["sel_idx", "mat_path", "method"] if "method" in df_eval.columns else ["sel_idx", "mat_path"],
    title="Explanation plausibility (Lenient AttAUC) — AF vs SNR",
)
fig1.update_layout(yaxis_title="Lenient AttAUC", xaxis_title="Class")
fig1.write_html(OUTDIR / "01_attauc_lenient_box.html")
_maybe_write_png(fig1, OUTDIR / "01_attauc_lenient_box.png")

# -----------------------------
# PLOT 2 — Faithfulness gain
# -----------------------------
fig2 = px.box(
    df_eval.dropna(subset=["faithfulness_gain"]),
    x="class_short",
    y="faithfulness_gain",
    points="all",
    category_orders={"class_short": class_order},
    hover_data=["sel_idx", "mat_path", "method"] if "method" in df_eval.columns else ["sel_idx", "mat_path"],
    title="Faithfulness (Gain over random deletion) — AF vs SNR",
)
fig2.update_layout(yaxis_title="Faithfulness gain (higher = better)", xaxis_title="Class")
fig2.write_html(OUTDIR / "02_faithfulness_gain_box.html")
_maybe_write_png(fig2, OUTDIR / "02_faithfulness_gain_box.png")

# -----------------------------
# PLOT 3 — Stability (extra_mid Spearman)
# -----------------------------
fig3 = px.box(
    df_stab.dropna(subset=["extra_mid_spearman"]),
    x="class_short",
    y="extra_mid_spearman",
    points="all",
    category_orders={"class_short": class_order},
    hover_data=["sel_idx", "mat_path", "extra_mid_k_eff"] if "extra_mid_k_eff" in df_stab.columns else ["sel_idx", "mat_path"],
    title="Stability under extra beat (middle) — Spearman (AF vs SNR)",
)
fig3.update_layout(yaxis_title="Spearman (ranking stability)", xaxis_title="Class")
fig3.write_html(OUTDIR / "03_stability_extra_mid_spearman_box.html")
_maybe_write_png(fig3, OUTDIR / "03_stability_extra_mid_spearman_box.png")

# -----------------------------
# PLOT 4 — Stability (extra_mid RBO)
# -----------------------------
fig4 = px.box(
    df_stab.dropna(subset=["extra_mid_rbo"]),
    x="class_short",
    y="extra_mid_rbo",
    points="all",
    category_orders={"class_short": class_order},
    hover_data=["sel_idx", "mat_path", "extra_mid_k_eff"] if "extra_mid_k_eff" in df_stab.columns else ["sel_idx", "mat_path"],
    title="Stability under extra beat (middle) — RBO (AF vs SNR)",
)
fig4.update_layout(yaxis_title="RBO (top-weighted ranking overlap)", xaxis_title="Class")
fig4.write_html(OUTDIR / "04_stability_extra_mid_rbo_box.html")
_maybe_write_png(fig4, OUTDIR / "04_stability_extra_mid_rbo_box.png")

print(f"Saved plots to: {OUTDIR.resolve()}")


Saved plots to: C:\UHull\ecg-xai\notebook\outputs\plots_af_snr
